# Прогнозирование обводнённости скважин с помощью методов машинного обучения

Version 2.3 final

Импорт необходимых библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import seaborn as sns
import pylab
from pylab import rcParams
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.metrics.pairwise import euclidean_distances

Загружаем исходные данные на 01.06.2018 из экселя

In [ ]:
data_path = '../../data/ps_owc/df.xlsx'
df = pd.read_excel(data_path, sheet_name='v1')

In [ ]:
df

Проверяем местоположение скважин - строим карту забоев

In [ ]:
ax = df.plot(kind='scatter', x='x', y='y')
df[['x','y','Well']].apply(lambda row: ax.text(*row),axis=1);
rcParams['figure.figsize'] = [11, 8]

# Конструирование признаков (feature engineering)

Рассчитываем матрицу евклидовых расстояний между скважинами из их координат

In [ ]:
distance = pd.DataFrame(euclidean_distances(df[['x', 'y']]))
distance

Извлекаем список имён скважин. Присваеваем имена скважин колонкам матрицы расстояний. Таким образом именуем столбцы с расстояними именами скважин, расстояния до которых вычислены.

In [ ]:
well_names = df['Well']
distance.columns = well_names
distance

Объединяем датасет парметров работы скважин с матрицей расстояний между скважинами. Таким образом в датасет добавляем N-столбцов, где N - количество скважин в датасете. Т.е. конструируем N новых признаков (feature engineering) где параметр удалённости является весом (вкладом/влиянием) скважин друг на друга.

In [ ]:
df_distance = pd.concat([df.drop(['x', 'y'], axis=1), distance], axis=1)
df_distance

# Проверка гипотезы

Создаём тренировочный дата сет, удаляя из него скважины, выбранные для теста и прогноза

In [ ]:
df_train_1 = df_distance.drop([12, 13, 14, 15], axis=0)
df_train_1

Создаём тестовый датасет

In [ ]:
df_test_1 = df_distance.loc[[12, 13]]
df_test_1

Создаём тренировочный DataFrame признаков X_1. Удаляем категорийный признак (имя скважины) и предсказываемое значение wct.

In [ ]:
x_1 = df_train_1.drop(['Well', 'wct'], axis=1)
x_1

Создаём тренировочный вектор целевых значений y_1

In [ ]:
y_1 = df_train_1['wct']
y_1

Создаём тестовый вектор целевых значений y_test_1

In [ ]:
y_test_1 = df_test_1['wct']
y_test_1

- Создаём скейлер для масштабирования данных
- обучаем скейлер на тренировочных данных и масштабируем их
- масштабируем тестовые данные на обученном скейлере
- создаём модель LinearRegression / RandomForestRegressor
- тренируем модель
- рассчитываем обводнённость по тестовой выборке
- рассчитываем обводнённость по тренировочной выборке
- оцениваем качество модели: считаем коэффициенты R^2 по тестовым и тренировочным выборкам

In [ ]:
scaler = StandardScaler()
x_train_1 = scaler.fit_transform(x_1)

x_test_1 = scaler.transform(df_test_1.drop(['Well', 'wct'], axis=1))

model = RandomForestRegressor(random_state=42, max_depth=14)
model.fit(x_train_1, y_1)

y_pred_train_1 = model.predict(x_train_1)
y_pred_1 = model.predict(x_test_1)

print('Predicted values from train data:')
r2_train = r2(y_1, y_pred_train_1)
mae_train = mae(y_1, y_pred_train_1)
mse_train = mse(y_1, y_pred_train_1)
print(f'R2 train: {r2_train.round(4)}')
print(f'MAE train: {mae_train.round(4)}')
print(f'MSE train: {mse_train.round(4)}')

print('Predicted values from test data (blind test / validation, the model has not seen this data):')
r2_test = r2(y_test_1, y_pred_1)
mae_test = mae(y_test_1, y_pred_1)
mse_test = mse(y_test_1, y_pred_1)
print(f'R2 test: {r2_test.round(4)}')
print(f'MAE test: {mae_test.round(4)}')
print(f'MSE test: {mse_test.round(4)}')

model

R2 метрика на тренировочной метрике превышает R2 на тестовой на 7%, что что означает низкую степень переобучения модели

Сравним предсказанную обводнённсть с фактической на тестовой выборке, которая не использовалась при обучении модели (blind test)

In [ ]:
df_y_test = pd.DataFrame({'Well': df_test_1['Well'], 
                          'wct predicted, %': y_pred_1.round(1), 
                          'wct actual, %': y_test_1.round(1),
                          'difference': (y_pred_1 - y_test_1).round(1)})
df_y_test

Сравним предсказанную обводнённость с фактической на тренировочной выборке

In [ ]:
df_y_train = pd.DataFrame({'Well': df_train_1['Well'], 
                           'wct predicted, %': y_pred_train_1.round(1), 
                           'wct actual, %': y_1.round(1),
                           'difference': (y_pred_train_1 - y_1).round(1)})
df_y_train

## Создание модели на всех доступных данных (train + test)

Создаём тренировочный дата сет, удаляя из него скважины, выбранные для прогноза

In [ ]:
df_train_2 = df_distance.drop([14, 15], axis=0)
df_train_2

Создаём датасет для прогнозирования из скважин, удалённых на предыдущем шаге.

Предсказываемый параметр WCT (обводнённсть) сейчас = NaN.

In [ ]:
df_fc = df_distance.loc[[14, 15]]
df_fc

В следующей ячейке представлена возможность смены кровли Ливенского горизонта в точке вскрытия скважины. Опция добавлена для ручного изменения параметра, чтобы оценить чувствительность модели к изменению параметра.

Чувствительность модели к параметру кровли структуры (принимаем, что кровля = верху интервала перфорации/открытого ствола) низкая. Изменение кровли пласта выше разумного предела пределах (>100 м) не приводит к значимому изменению обводённсти.

In [ ]:
# Исходный параметр Top perf = -2314 м а.о.
# df_fc.at[15,'Top perf']= -2300
# df_fc

Создаём тренировочный DataFrame признаков x_2. Удаляем категорийный признак (имя скважины) и предсказываемое значение wct.

In [ ]:
x_2 = df_train_2.drop(['Well', 'wct'], axis=1)
x_2

Создаём тренировочный вектор целевых значений y_2

In [ ]:
y_2 = df_train_2['wct']
y_2

In [ ]:
scaler = StandardScaler()
x_train_2 = scaler.fit_transform(x_2)

x_fc = scaler.transform(df_fc.drop(['Well', 'wct'], axis=1))

model = RandomForestRegressor(random_state=42, max_depth=14)
model.fit(x_train_2, y_2)

y_pred_train_2 = model.predict(x_train_2)
y_fc = model.predict(x_fc)

print('Predicted values from train data:')
r2_train = r2(y_2, y_pred_train_2)
mae_train = mae(y_2, y_pred_train_2)
mse_train = mse(y_2, y_pred_train_2)
print(f'R2 train: {r2_train.round(4)}')
print(f'MAE train: {mae_train.round(4)}')
print(f'MSE train: {mse_train.round(4)}')

print('Forecasted values could be compared with real data!')

model

R2 повысилось. Или модель переобучилась или большее количество данных помогло точнее настроить модель

Сравним предсказанную обводнённость с фактической на тренировочной выборке.

In [ ]:
df_y_train = pd.DataFrame({'Well': df_train_2['Well'], 
                           'wct predicted, %': y_pred_train_2.round(1), 
                           'wct actual, %': y_2.round(1),
                           'difference': (y_pred_train_2 - y_2).round(1)})
df_y_train

Предсказываемя обводнённость по боковым стволам:

In [ ]:
df_y_test = pd.DataFrame({'Well': df_test_1['Well'], 
                          'wct predicted, %': y_pred_1.round(1), 
                          'wct actual, %': y_test_1.round(1),
                          'difference': (y_pred_1 - y_test_1).round(1)})
df_y_test

Выводим список признаков в порядке убыввания их важности

In [ ]:
model.feature_importances_
feature_importances = pd.DataFrame()
feature_importances['feature_name'] = x_2.columns.tolist()
feature_importances['importance'] = model.feature_importances_
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

Строим диаграмму важности признаков

In [ ]:
fig = px.bar(feature_importances, 
             x=feature_importances['importance'], 
             y=feature_importances['feature_name'], 
             title="Feature importances")
fig.show()

Сравнительный график реальных и предсказанных значений.

In [ ]:
fig = px.scatter(x=y_pred_train_2, y=y_2, title="True vs Predicted values",
                 text=df_train_2['Well'], width=850, height=800)
fig.add_trace(go.Scatter(x=[0,100], y=[0,100], mode='lines', name='True=Predicted',
                         line = dict(color='red', width=1, dash='dash')))
fig.update_xaxes(title_text='Predicted')
fig.update_yaxes(title_text='True')
fig.show()